In [2]:
import os, sys
import numpy as np
import time
import matplotlib.pyplot as plt
import xarray as xr
from pathlib import Path

from generate_sim_data import gen_rho, gen_sim_data

In [101]:
home = str(Path.home())
save_dir = home + r'\OneDrive - UCB-O365\ARSENL\Experiments\SPCM\Data\Simulated\manuscript_revise_distributed'

# amplitude = np.logspace(7.1, 10, 20)
amplitude = np.array([1e8, 3e8, 6e8, 1.2e9])  # [Hz]
# amplitude = np.array([1e8])

# simulation resolution settings
t_sim_min = 0  # [s]
t_sim_max = 40e-9  # [s]
dt_sim = 25e-12  # [s]

tD = 25e-9  # [s] deadtime
Nshot = int(1e3)  # number of laser shots
wrap_deadtime = True  # wrap deadtime between shots
window_bnd = np.array([28e-9, 34e-9])  # [s] time-of-flight bounds
laser_pulse_width = 5e-10  # [s] laser pulse width (sigma) 
target_time = 31.2e-9  # [s] target location in time
background = 1e1  # [Hz] background count rate (dark count)

In [102]:
# To iterate at different widths and scales
scale = 10
target_time *= scale  # [s]
laser_pulse_width *= scale  # [s] laser pulse width (sigma) 
window_bnd *= scale  # [s] time-of-flight bounds
t_sim_max *= scale  # [s]
dt_sim *= scale  # [s]

In [103]:
for i in range(len(amplitude)):
    
    target_amplitude = amplitude[i]
    
    ### GENERATE SIMULATED DATA ###
    det_idx, phot_idx, sync_idx, det_sync_idx, phot_sync_idx, det_events, phot_events = gen_sim_data(t_sim_max, dt_sim,
                                                                                                     tD, Nshot,
                                                                                                     wrap_deadtime,
                                                                                                     window_bnd,
                                                                                                     laser_pulse_width,
                                                                                                     target_time,
                                                                                                     target_amplitude,
                                                                                                     background)

    time_tag_index = det_idx
    true_time_tag_index = phot_idx
    sync_index = sync_idx
    time_tag = det_events
    true_time_tag = phot_events
    time_tag_sync_index = det_sync_idx
    true_time_tag_sync_index = phot_sync_idx

    # Save simulated data to netCDF
    sim_data = xr.Dataset(
        data_vars=dict(
            time_tag=(['time_tag_index'], time_tag),
            time_tag_sync_index=(['time_tag_index'], time_tag_sync_index),
            true_time_tag=(['true_time_tag_index'], true_time_tag),
            true_time_tag_sync_index=(['true_time_tag_index'], true_time_tag_sync_index),
            laser_pulse_width=laser_pulse_width,
            target_time=target_time,
            target_amplitude=target_amplitude,
            background=background,
            window_bnd=window_bnd,
            dt_sim=dt_sim
        ),
        coords=dict(
            sync_index=(['sync_index'], sync_index)
        )
    )

    fname = r'\sim_amp{:.1E}_nshot{:.1E}_width{:.1E}_dt{:.1E}.nc'.format(target_amplitude, Nshot, laser_pulse_width, dt_sim)
    sim_data.to_netcdf(save_dir+fname)
    
print('(First 10) Time tags:', time_tag[:10])

time elapsed: 0.7128558158874512
time elapsed: 2.322800397872925
time elapsed: 3.907776355743408
time elapsed: 6.447546482086182
(First 10) Time tags: [1220.665454976914, 1221.573215647288, 1217.2597721045408, 1210.8178897900839, 1219.31892928682, 1215.8083629338714, 1229.4494244145867, 1215.505443819943, 1219.9615088516466, 1196.1079334823592]
